Задание 5. Web-scraping на языке Python

Инструкция:

Шаг 1.  Изучите материалы лекционных и практических занятий по темам 4.1 и 4.2.

Шаг 2. Используя доступ к данным по API, соберите с [ресурса](https://trudvsem.ru/) данные по интересующим Вас вакансиям или вакансиям Вашего региона.
Дополнительную информацию  по данному API можно найти [здесь](https://trudvsem.ru/opendata/api). Количество записей и набор признаков определяете самостоятельно. Собранные данные сохраните в DataFrame.  

Шаг 3. С использованием библиотеки BeautifulSoup выполните web-scraping [ресурса](https://prodoctorov.ru/). Населенный пункт и специализацию врача выбрать самостоятельно.  Количество записей и набор признаков определяете самостоятельно. Собранные данные сохраните в DataFrame.

Шаг 4.  Опубликуйте файл расширения ipynb на платформе Odin.

In [68]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
import re
from math import nan

reStars = re.compile(r'width:\s*(\d+)\s*px')
reExp = re.compile(r'Стаж\s*(\d+)\s*(лет|года|год)')
reWs = re.compile(r'\s+')

def nested_get(d: dict, path: str):
    path = path.split('.')
    node = d
    for attr in path:
        if attr in node:
            node = node.get(attr)  # either node or value
        else:
            return None  # attribute not found
    return(node)

def my_toint(val):
    if val == 'от 0':
        return 0
    else:
        return int(val)

def fails_toint(val):
    try:
        my_toint(val)
    except:
        return True
    return False

In [32]:
url = r'http://opendata.trudvsem.ru/api/v1/vacancies/'
response = requests.get(url, {'region_code': 47, 'offset': 1, 'limit': 100})
response.status_code

200

In [46]:
COI = ['job-name', 'creation-date', 'category.specialisation', 'salary_min', 'requirement.experience', 'requirement.education']
record_limit = 100
page_limit = 300

table = []
pageNo = 0
while True:
    response = requests.get(url, {'region_code': 47, 'offset': pageNo, 'limit': record_limit}, timeout=10)
    assert response.status_code == requests.codes.ok, "bad status code (%d)@page %d"%(request.status_code, pageNo)    
    
    vacancies = (response.json())['results']['vacancies']
    vac_tbl = [{coi: nested_get(vacancy, 'vacancy.%s'%coi) for coi in COI} for vacancy in vacancies]
    table.extend(vac_tbl)
    if len(vac_tbl) < record_limit:
        break
    if pageNo >= page_limit-1:
        break
    
    print('got page #%d'%pageNo)
    pageNo += 1
#     sleep(0.1)
print('done processing data source')

got page #0
got page #1
got page #2
got page #3
got page #4
got page #5
got page #6
got page #7
got page #8
got page #9
got page #10
got page #11
got page #12
got page #13
got page #14
got page #15
got page #16
got page #17
got page #18
got page #19
got page #20
got page #21
got page #22
got page #23
got page #24
got page #25
got page #26
got page #27
got page #28
got page #29
got page #30
got page #31
got page #32
got page #33
got page #34
got page #35
got page #36
got page #37
got page #38
got page #39
got page #40
got page #41
got page #42
got page #43
got page #44
got page #45
got page #46
got page #47
got page #48
got page #49
got page #50
got page #51
got page #52
got page #53
got page #54
got page #55
got page #56
got page #57
got page #58
got page #59
got page #60
got page #61
got page #62
got page #63
got page #64
got page #65
got page #66
got page #67
got page #68
got page #69
got page #70
got page #71
got page #72
got page #73
got page #74
got page #75
got page #76
got page 

In [80]:
df_api = pd.DataFrame(table)
df_api['creation-date'] = pd.to_datetime(df_api['creation-date'])
df_api['requirement.experience'] = df_api['requirement.experience'].apply(my_toint)  # может содержать "от 0"
# df_api['requirement.education'].unique()  # to Categorical?
df_api.info()
# df_api[df_api['job-name'].str.lower().str.contains('мастер') & (df_api.salary_min > 0)].nsmallest(10, 'salary_min')
# df_api.nsmallest(10, 'creation-date')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11510 entries, 0 to 11509
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   job-name                 11510 non-null  object        
 1   creation-date            11510 non-null  datetime64[ns]
 2   category.specialisation  11510 non-null  object        
 3   salary_min               11510 non-null  int64         
 4   requirement.experience   11510 non-null  int64         
 5   requirement.education    11510 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 539.7+ KB


In [3]:
url2 = r'https://prodoctorov.ru/spb/genetik/'
response = requests.get(url2)   # не отправляет метаданные характерные для браузера(
response.status_code

<Response [200]>

In [53]:
responses = []
for pageNo in range(1, 5):
    responses.append(requests.get(url2, params={'page': pageNo}))
#     sleep(10)
    print(pageNo)

1
2
3
4


In [58]:
# место чтобы чинить побитые защитой от ботов ответы
# надо зайти на сайт браузером и решить капчу, тогда защита успокоится на пару раз
# вместо n ввести номер ответа с проблемой, выполнить, повторять до готовности

n = 4
responses[n-1] = requests.get(url2, params={'page': n})
print(*map(lambda r: str(r.text)[100:500], responses))

ef="https://mc.yandex.ru"
        ><link
            rel="preconnect"
            href="https://mc.yandex.ru"
        ><meta
            http-equiv="Content-Type"
            content="text/html; charset=UTF-8"
        ><meta
            name="viewport"
            content="width=1000"
        ><link
                rel="canonical"
                href="https://prodoctorov.ru/spb/genetik/"
         ef="https://mc.yandex.ru"
        ><link
            rel="preconnect"
            href="https://mc.yandex.ru"
        ><meta
            http-equiv="Content-Type"
            content="text/html; charset=UTF-8"
        ><meta
            name="viewport"
            content="width=1000"
        ><link
                rel="canonical"
                href="https://prodoctorov.ru/spb/genetik/?page=2"
  ef="https://mc.yandex.ru"
        ><link
            rel="preconnect"
            href="https://mc.yandex.ru"
        ><meta
            http-equiv="Content-Type"
            content="text/html; cha

In [91]:
# import pickle   # дополнительно сохраню готовые отклики сервера. Раскомментируй и запусти чтобы восстановить
# # with open('./responses.pkl', 'wb') as hFile:
# #     pickle.dump(responses, hFile)
# with open('./responses.pkl', 'rb') as hFile:  # файл должен лежать в той же папке, что и ipynb
#     responses = pickle.load(hFile)

In [83]:
#key, tag name filter, tag attributes filter
COI = [('doc_name', 'span', {'class':'b-doctor-card__name-surname'}),
      ('specialization', 'div', {'class':"b-doctor-card__spec"}),
      ('job_experience', 'div', {'class', 'ui-text ui-text_subtitle-1'}),
      ('*category', 'span', {'class': 'ui-text ui-text_body-2 b-doctor-card__text-with-dot'}),
      ('?warning', 'div', {'class': 'ui-text ui-text_body-2 b-doctor-card__warning-text'}),
      ('?papers_ok', 'span', {'class': 'ui-icon-shield-check ui-icon_fz_smaller mr-1'})
#       ('fee', 'div', {'class':'ui-text ui-text_subtitle-1'}),  # dynamically-generated
      ]
width_of_5stars = 135  # in px

table_raw = []
for response in responses:
    parsedData = BeautifulSoup(response.text)
    cardElems = parsedData.find_all('div', class_='b-doctor-card')
    for element in cardElems:
        elemdata = dict()
        for key, tagName, tagAttr in COI:
            results = element.find_all(name=tagName, attrs=tagAttr)
            if len(results) <= 0:
                if key[0] in['?', '*']:
                    continue
                else:
                    raise AssertionError('tag not found @element '+str(element))

            if len(results) > 1:
                if key[0] in['+', '*']:
                    elemdata[key[1:]] = [r.get_text(strip=True) for r in results]
                else:
                    raise AssertionError('found multiple '+key+' tags @element '+str(element))
            else:
                if key[0] in ['?', '*']:
                    key = key[1:]
                elemdata[key] = results[0].get_text(strip=True)

        starstring = element.find('div', class_='b-stars-rate__progress')['style']
        match = reStars.fullmatch(starstring)
        elemdata['stars'] = 5*int(match[1]) / width_of_5stars

        elemFeeContainer = element.find('doctor-price-block')
        if elemFeeContainer is not None:
            elemdata['fee'] = float(elemFeeContainer[':price'])
    #     elemdata['job_experience'] = int()
        table_raw.append(elemdata)
table_raw

[{'doc_name': 'Поленникова Эльвира Сергеевна',
  'specialization': 'Генетик',
  'job_experience': 'Стаж 14 лет',
  'papers_ok': '',
  'stars': 4.222222222222222,
  'fee': 4600.0},
 {'doc_name': 'Шабанова Елена Сергеевна',
  'specialization': 'Генетик',
  'job_experience': 'Стаж 16 лет',
  'category': 'высшая категория',
  'papers_ok': '',
  'stars': 4.7407407407407405,
  'fee': 3900.0},
 {'doc_name': 'Королева Валерия Дмитриевна',
  'specialization': 'Детский невролог, \n                    \n                \n                    \n                        генетик',
  'job_experience': 'Стаж 5 лет',
  'papers_ok': '',
  'stars': 3.6666666666666665,
  'fee': 3800.0},
 {'doc_name': 'Двойнова Наталья Михайловна',
  'specialization': 'Генетик',
  'job_experience': 'Стаж 8 лет',
  'papers_ok': '',
  'stars': 3.4444444444444446,
  'fee': 3800.0},
 {'doc_name': 'Кречмар Марина Валерьевна',
  'specialization': 'Генетик',
  'job_experience': 'Стаж 35 лет',
  'category': 'высшая категория',
  'st

In [84]:
def normalize_experience(exp_str: str) -> float:
    if exp_str.lower() == 'нет данных о стаже':
        return nan
    else:
        return float(reExp.fullmatch(exp_str)[1])

def normalize_category(cat_object: [list, str]) -> str:
    if isinstance(cat_object, list):
        return ', '.join(cat_object)
    else:
        return cat_object

def normalize_papers(papers_str):
    return True  # true либо NaN

def normalize_specialization(spec_str: str) -> str:
    return reWs.sub(' ', spec_str)

norm_methods = {'job_experience': normalize_experience, 'category': normalize_category, 'papers_ok': normalize_papers,
               'specialization': normalize_specialization}
table = []
for record in table_raw:
    norm_record = record.copy()
#     print(record)
    for attr, meth in norm_methods.items():
        if attr in norm_record:
            norm_record[attr] = meth(norm_record[attr])
    table.append(norm_record)

In [101]:
df2 = pd.DataFrame(table)
df2.info()
# df2.nlargest(5, 'job_experience')
# df2[df2.warning.notna()]
# df2.category.unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   doc_name        69 non-null     object 
 1   specialization  69 non-null     object 
 2   job_experience  61 non-null     float64
 3   papers_ok       21 non-null     object 
 4   stars           69 non-null     float64
 5   fee             17 non-null     float64
 6   category        26 non-null     object 
 7   warning         3 non-null      object 
dtypes: float64(3), object(5)
memory usage: 4.4+ KB
